***GENERATED CODE FOR autofedate PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, unix_timestamp
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ConvertToDate(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]
    dateFormate = transformationData["dateFormat"]
    dfReturn = dfReturn.withColumn(feature, unix_timestamp(feature, dateFormate)
                                   .cast("double").cast("timestamp"))
    dfReturn = dfReturn.withColumn(
        feature + '_dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ConvertToDate(transformationDF, {'transformationsData': [{'feature_label': 'collection_week', 'dateFormat': 'MM-dd-yyyy', 'transformation_label': 'Convert to Date'}], 'feature': 'collection_week', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                         'count': '74', 'mean': '', 'stddev': '', 'min': '01-13-23', 'max': '12-24-21', 'missing': '0', 'distinct': '57'}, 'transformation': [{'transformation': 'Convert to Date', 'selectedAsDefault': 1}], 'updatedLabel': 'collection_week'}, {'feature_label': 'collection_week', 'dateFormat': 'MM-dd-yyyy', 'transformation_label': 'Convert to Date'})
        transformationDF = transformationDF.drop('collection_week')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '74', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '74', 'mean': '0.0', 'stddev': 'nan', 'min': '0', 'max': 'WYNNE', 'missing': '0', 'distinct': '69'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'hospital_subtype', 'transformation_label': 'String Indexer'}], 'feature': 'hospital_subtype', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '74', 'mean': '', 'stddev': '', 'min': 'Childrens Hospitals', 'max': 'Short Term', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'hospital_subtype'}, {'feature_label': 'hospital_subtype', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('hospital_subtype')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run autofedateHooks.ipynb
try:
	#sourcePreExecutionHook()

	coronatstrainuhkr = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1717507946723_CoronaTSTrain.csv', 'filename': 'CoronaTSTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/healthcare/CoronaHospitalCapability/datafiles/CoronaTSTrain.csv', 'viewFileName': 'CoronaTSTrain.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run autofedateHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(coronatstrainuhkr,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "collection_week", "dateFormat": "MM-dd-yyyy", "transformation_label": "Convert to Date"}], "feature": "collection_week", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "74", "mean": "", "stddev": "", "min": "01-13-23", "max": "12-24-21", "missing": "0", "distinct": "57"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "collection_week"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "74", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "74", "mean": "0.0", "stddev": "nan", "min": "0", "max": "WYNNE", "missing": "0", "distinct": "69"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "hospital_subtype", "transformation_label": "String Indexer"}], "feature": "hospital_subtype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "74", "mean": "", "stddev": "", "min": "Childrens Hospitals", "max": "Short Term", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "hospital_subtype"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fips_code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "26129.59", "stddev": "15669.75", "min": "0", "max": "56023", "missing": "0"}, "updatedLabel": "fips_code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_metro_micro", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "0.78", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "is_metro_micro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "74", "mean": "118.38", "stddev": "157.19", "min": "0.0", "max": "669.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_beds_7_day_avg"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_avg", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "74", "mean": "10.09", "stddev": "26.64", "min": "0.0", "max": "203.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "802.74", "stddev": "1090.6", "min": "0", "max": "4687", "missing": "0"}, "updatedLabel": "total_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "688.84", "stddev": "1013.02", "min": "0", "max": "4421", "missing": "0"}, "updatedLabel": "all_adult_hospital_beds_7..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "all_adult_hospital_inpatient_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "741.77", "stddev": "927.75", "min": "0", "max": "4421", "missing": "0"}, "updatedLabel": "all_adult_hospital_inpati..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_used_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "73.95", "stddev": "185.5", "min": "0", "max": "1422", "missing": "0"}, "updatedLabel": "inpatient_beds_used_covid..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "70.31", "stddev": "182.76", "min": "0", "max": "1392", "missing": "0"}, "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_adult_patients_hospitalized_confirmed_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "65.72", "stddev": "175.31", "min": "0", "max": "1356", "missing": "0"}, "updatedLabel": "total_adult_patients_hosp..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "inpatient_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "851.3", "stddev": "1003.2", "min": "0", "max": "4582", "missing": "0"}, "updatedLabel": "inpatient_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_icu_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "119.09", "stddev": "201.38", "min": "0", "max": "1054", "missing": "0"}, "updatedLabel": "total_icu_beds_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_staffed_adult_icu_beds_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "84.27", "stddev": "149.64", "min": "0", "max": "812", "missing": "0"}, "updatedLabel": "total_staffed_adult_icu_b..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "icu_beds_used_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "81.8", "stddev": "155.93", "min": "0", "max": "877", "missing": "0"}, "updatedLabel": "icu_beds_used_7_day_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "16.57", "stddev": "55.76", "min": "0", "max": "453", "missing": "0"}, "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "staffed_icu_adult_patients_confirmed_covid_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "16.07", "stddev": "55.1", "min": "0", "max": "453", "missing": "0"}, "updatedLabel": "staffed_icu_adult_patient..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous_day_covid_ED_visits_7_day_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "74", "mean": "80.66", "stddev": "144.42", "min": "0", "max": "617", "missing": "0"}, "updatedLabel": "previous_day_covid_ED_vis..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run autofedateHooks.ipynb
try:
	#sinkPreExecutionHook()

	datapipefortsoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/666292198a1ee41285a40f3d/part-00000-741990f8-4b8a-4011-b0aa-c156309bb936-c000.csv', 'samplecount': 742, 'originalcount': 742, 'orignalKey': '/FileStore/platform/testdata/1717507946723_CoronaTSTrain.csv', 'pathOnly': '/healthcare/CoronaHospitalCapability/datapipeforTS', 'project_id': '665993e56e3600cfb7189b4f', 'parent_id': '665993e56e3600cfb7189b4f', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41084', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41085', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41086', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41087', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41088', 'field': 'total_beds_7_day_sum', 'alias': 'total_beds_7_day_sum', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41089', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108a', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108b', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108c', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108d', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108e', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4108f', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41090', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41091', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41092', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41093', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41094', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41095', 'field': 'state_stringindexer', 'alias': 'state_stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41096', 'field': 'city_stringindexer', 'alias': 'city_stringindexer', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41097', 'field': 'hospital_subtype_stringindexer', 'alias': 'hospital_subtype_stringindexer', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41098', 'field': 'collection_week_converttodate', 'alias': 'collection_week_converttodate', 'type': 'date', 'position': '20'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4106f', 'field': 'collection_week', 'alias': 'collection_week', 'type': 'date', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41070', 'field': 'state', 'alias': 'state', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41071', 'field': 'city', 'alias': 'city', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41072', 'field': 'hospital_subtype', 'alias': 'hospital_subtype', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41073', 'field': 'fips_code', 'alias': 'fips_code', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41074', 'field': 'is_metro_micro', 'alias': 'is_metro_micro', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41075', 'field': 'total_beds_7_day_avg', 'alias': 'total_beds_7_day_avg', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41076', 'field': 'inpatient_beds_used_covid_7_day_avg', 'alias': 'inpatient_beds_used_covid_7_day_avg', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41077', 'field': 'total_beds_7_day_sum', 'alias': 'total_beds_7_day_sum', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41078', 'field': 'all_adult_hospital_beds_7_day_sum', 'alias': 'all_adult_hospital_beds_7_day_sum', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41079', 'field': 'all_adult_hospital_inpatient_beds_7_day_sum', 'alias': 'all_adult_hospital_inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107a', 'field': 'inpatient_beds_used_covid_7_day_sum', 'alias': 'inpatient_beds_used_covid_7_day_sum', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107b', 'field': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107c', 'field': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'alias': 'total_adult_patients_hospitalized_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107d', 'field': 'inpatient_beds_7_day_sum', 'alias': 'inpatient_beds_7_day_sum', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107e', 'field': 'total_icu_beds_7_day_sum', 'alias': 'total_icu_beds_7_day_sum', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a4107f', 'field': 'total_staffed_adult_icu_beds_7_day_sum', 'alias': 'total_staffed_adult_icu_beds_7_day_sum', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41080', 'field': 'icu_beds_used_7_day_sum', 'alias': 'icu_beds_used_7_day_sum', 'type': 'numeric', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41081', 'field': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41082', 'field': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'alias': 'staffed_icu_adult_patients_confirmed_covid_7_day_sum', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '666292fa8a1ee41285a41083', 'field': 'previous_day_covid_ED_visits_7_day_sum', 'alias': 'previous_day_covid_ED_visits_7_day_sum', 'type': 'numeric', 'position': '20'}], 'server': 'https://dnm.bfirst.ai:443', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'resultforAutoFEDate.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-580e152f-06be-48c5-b60a-392e4fda1b80-c000.csv', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
